In [1]:
!cd /Users/lc/Developer/kuzu &&  make clean-python-api && make release NUM_THREADS=20 

rm -rf tools/python_api/build
cd external && \
	mkdir -p build && \
	cd build && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../ && \
	cmake --build . --config Release -- -j 20
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/lc/Developer/kuzu/external/build
[100%] Built target apache_arrow
mkdir -p build/release && \
	cd build/release && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../.. && \
	cmake --build . --config Release -- -j 20
-- pybind11 v2.11.0 dev1
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/lc/Developer/kuzu/build/release
[  1%] Built target utf8proc
[  2%] Built target kuzu_transaction
[  2%] Built target kuzu_common_csv_reader
[  3%] Built target kuzu_common_data_chunk
[  3%] Built target kuzu_common_task_system
[  3%] Built target kuzu_catalog
[  3%] Built target kuzu_common_arrow
[  4%] Built target kuzu_binder_expression
[  5%] Built target kuzu_common_vector
[  6%] Built target kuzu_binder
[  7%] Built tar

In [2]:
import sys
sys.path.append('/Users/lc/Developer/kuzu/tools/python_api/build')
import kuzu

In [3]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG


dataset = OGB_MAG(root='/Users/lc/Developer/ogb_pytorch', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

In [4]:
import torch

In [5]:
DB_PATH = '/Users/lc/Developer/kuzu_ogb_mag'
db = kuzu.Database(DB_PATH)

In [6]:
conn = kuzu.Connection(db)

In [7]:
fs, gs = db.get_torch_geometric_remote_backend()

In [8]:
from torch_geometric.loader import NeighborLoader


In [9]:
loader_kuzu = NeighborLoader(
    data=(fs, gs),
    num_neighbors={key.edge_type : [8] * 2 for key in gs.get_all_edge_attrs()},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=128,
    input_nodes=('paper', torch.arange(0, data['paper'].num_nodes)),
    input_time=None
)

In [10]:
import time
times = []
for i in range(10):
    start = time.time()
    next(iter(loader_kuzu))
    end = time.time()
    if i > 0:
        times.append(end - start)
        print(end - start)
print("=====")
print(sum(times) / len(times))

28.71490478515625
28.46481680870056
27.981308937072754
27.89212918281555
28.72823977470398
28.5123131275177
28.577126264572144
28.54621911048889
28.4129798412323
=====
28.425559759140015


In [11]:
loader_pyg = NeighborLoader(
    data=data,
    num_neighbors={key.edge_type : [8] * 2 for key in gs.get_all_edge_attrs()},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=128,
    input_nodes=('paper', torch.arange(0, data['paper'].num_nodes)),
    input_time=None
)

In [12]:
import time
times = []
for i in range(10):
    start = time.time()
    next(iter(loader_pyg))
    end = time.time()
    if i > 0:
        times.append(end - start)
        print(end - start)
print("=====")
print(sum(times) / len(times))

0.023946046829223633
0.02199387550354004
0.02047896385192871
0.023924827575683594
0.021136045455932617
0.02242588996887207
0.022296905517578125
0.02095818519592285
0.020947933197021484
=====
0.022012074788411457
